# 데이터 수집 및 로드

In [ ]:
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/데이터분석스쿨2기/share/12.데이터활용및과학방법론-12/한국어_연속형_대화_데이터셋_7감정.xlsx')
df.shape

(55628, 2)

In [ ]:
# 샘플 확인
df.head()

,발화,감정
0,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노
1,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오
2,손님 왔어요.,중립
3,손님? 누구?,중립
4,몰라요. 팀장님 친구래요.,중립


In [ ]:
# 실습 1분
# 결측 제거, 중복 제거
df.dropna(inplace=True)
df.drop_duplicates(subset=['발화'], inplace=True)
df.shape

(51232, 2)

In [ ]:
# 감정 타겟의 값 별(집계) 카운트 획득, 실습 1분
df.groupby('감정').count()

,발화
감정,
ㄴ중립,1
ㅈ중립,1
ㅍ,12
공포,95
놀람,3874
분,4
분ㄴ,1
분노,3536
슬픔,1848


In [ ]:
# 값 보정 실습 2분
# ㄴ중립, ㅈ중립,
# ㅍ
# 분, 분ㄴ
# 줄, 중림
def change( x ):
    # 데이터가 1개씩 전달
    if      x=='ㄴ중립' or x=='ㅈ중립' or x=='줄' or x=='중림' : return '중립'
    elif    x=='ㅍ': return '공포'
    elif    x=='분' or x=='분ㄴ':   return '분노'
    return x

df['감정'] = df['감정'].apply( change )
df.groupby('감정').count()

,발화
감정,
공포,107
놀람,3874
분노,3541
슬픔,1848
중립,40687
행복,983
혐오,192


- 데이터 대부분이 중립, 공포, 혐오등등 데이터 작음
    - 적절한 데이터는 아님
    - 현 노트는 워크플로우 중심으로 진행

In [ ]:
emotion_df = df.copy( True )
emotion_df.groupby('감정').count()

,발화
감정,
공포,107
놀람,3874
분노,3541
슬픔,1848
중립,40687
행복,983
혐오,192


In [ ]:
# 발화 -> 토크나이저를 통해 인코딩 처리
# 감정 -> 수치변환, 레이블인코더 적용 0, 1, 2, ~, .6 대체
# 레이블인코더 적용하여 감정 데이터는 수치로 변환 : 실습 2분
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
# 학습
encoder.fit(emotion_df['감정'])

# 변환
emotion_df['감정'] = encoder.transform( emotion_df['감정'] )
emotion_df.groupby('감정').count()

,발화
감정,
0,107
1,3874
2,3541
3,1848
4,40687
5,983
6,192


In [ ]:
# 분류값 -> 원래문구로 복원
encoder.inverse_transform( [ 5 ] )[0]

'행복'

In [ ]:
# 현재 데이터
emotion_df.head(2 )

,발화,감정
0,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,2
1,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,6


In [ ]:
# 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                emotion_df['발화'],
                                emotion_df['감정'],
                                random_state=0,
                                # [수정 1] 훈련 데이터는 50%만 처리 -> 학습종료를 빠르게 처리위함
                                train_size=0.5, stratify=emotion_df['감정'],
                                )

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((25616,), (25616,), (25616,), (25616,))

# 학습

- repo
    - 'klue/bert-base'
- 분류개수
    - 7 or len(emotion_df['감정'])

In [ ]:
len(emotion_df['감정'].unique())

7

In [ ]:
!pip install transformers -q

In [ ]:
# 학습(1 epoch 만)까지 진행
# 실습 6분
from transformers import BertTokenizerFast
from transformers import TFBertForSequenceClassification

repo      = 'klue/bert-base'
tokenizer = BertTokenizerFast.from_pretrained( repo )
model     = TFBertForSequenceClassification.from_pretrained( repo,
                                from_pt   =True,
                                num_labels=len(emotion_df['감정'].unique())
                                )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTo

In [ ]:
X_train = tokenizer( X_train.to_list() , truncation=True, padding=True )
X_test  = tokenizer( X_test.to_list(),   truncation=True, padding=True )

In [ ]:
print( X_train[0] )
print( X_train[0].tokens )
print( X_train[0].ids )

Encoding(num_tokens=141, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '상처', '##가', '났', '##네', '?', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

In [ ]:
y_train = y_train.tolist()
y_test  = y_test.tolist()

# 7개의 감정값
y_train[0]

1

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices( ( dict(X_train), y_train ) )
test_dataset  = tf.data.Dataset.from_tensor_slices( ( dict(X_test), y_test ) )

In [ ]:
'''
- 다중분류
    - CategoricalCrossentropy -> 정답이 원핫인코딩
    - SparseCategoricalCrossentropy -> 정답이 정수일대

=> 수정 : SparseCategoricalCrossentropy()
=> 데이터는 절반 이하로 양을 줄여서 진행
'''
model.compile(  optimizer   =tf.keras.optimizers.Adam(learning_rate=0.0001),
                loss        =tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics     =['accuracy']
            )

In [ ]:
# 체크
# with tf.device('device:GPU:0'):
#     model.fit( train_dataset.shuffle(1000).batch(128),
#                epochs       = 1, # 성능 감안하고 1회만 학습 -> 실제는 더 많은 학습 진행
#                batch_size   = 128,
#                )
# [수정 2]128 -> 64로 1회당 학습 사이즈 줄임 (메모리문제)
with tf.device('device:GPU:0'):
    model.fit( train_dataset.shuffle(1000).batch(64),
               epochs       = 1, # 성능 감안하고 1회만 학습 -> 실제는 더 많은 학습 진행
               batch_size   = 64,
               )

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


# 모델 저장

In [ ]:
# 실제로는 영속 저장 공간(구글 드라이브 등..) 저장
model.save_pretrained('multi_kobert')
tokenizer.save_pretrained('multi_kobert')

# 모델 로드

In [ ]:
!pip install transformers -q

In [ ]:
from transformers import BertTokenizerFast
from transformers import TFBertForSequenceClassification

save_dir = '/content/drive/MyDrive/데이터분석스쿨2기/share/12.데이터활용및과학방법론-12/spot'

tokenizer = BertTokenizerFast.from_pretrained( save_dir )
model     = TFBertForSequenceClassification.from_pretrained( save_dir )

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/데이터분석스쿨2기/share/12.데이터활용및과학방법론-12/spot.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
from transformers import TextClassificationPipeline

clf = TextClassificationPipeline(
    tokenizer           = tokenizer,
    model               = model,
    framework           = 'tf',
    return_all_scores   = True
)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# 성능은 배제, 처리 방식만 확인
y_pred = clf('내일은 주말이네요')[0]
sorted_y_pred = sorted( y_pred, key=lambda x:x['score'], reverse=True)
sorted_y_pred

[{'label': 'LABEL_5', 'score': 0.19520771503448486},
 {'label': 'LABEL_0', 'score': 0.19319069385528564},
 {'label': 'LABEL_2', 'score': 0.19080600142478943},
 {'label': 'LABEL_6', 'score': 0.1452164202928543},
 {'label': 'LABEL_3', 'score': 0.13637955486774445},
 {'label': 'LABEL_4', 'score': 0.08263397216796875},
 {'label': 'LABEL_1', 'score': 0.056565627455711365}]

In [ ]:
# 단 분류개수가 10개 이상이 되면 추출방식은 변경
clf_num = int(sorted_y_pred[0]['label'].split('_')[-1])
clf_num

5

In [ ]:
# 인코더 위에서 처리후 진행
encoder.transform( clf_num ) # 5번 기준이면 "행복"

- 차후과제    
    - 학습 장비를 확장 -> 대량 데이터 학습 필요
        - 모델 고도화
    - 텍스트 -> 감정분석 -> 스포티파이에 연동 (어떤 곡을 추천할것인가?)
        - 간단한 음원에 대한 피처(스포티파이를 통해 데이터 확보) 성분으로 구간을 체크해서 연결(진행)